In [4]:
import mlflow

# Set tracking URI (for example, a remote server or specific directory)
mlflow.set_tracking_uri('http://localhost:5000')

In [14]:
import mlflow
import mlflow.sklearn
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Set the MLflow experiment name
mlflow.set_experiment('house_price_prediction_experiment_With_regularization')

def train_and_log_experiment(model_name, params, run_name_param):
    try:
        # Load dataset
        df = pd.read_csv('D:/BITS-wiLP/Semester-3/MLOPs/Assignment/ML Model Deployment/ML Model Deployment/data/house_data.csv')
        X = df[['bedrooms', 'bathrooms', 'floors', 'yr_built']]
        y = df['price']
    except Exception as e:
        print(f"Error loading dataset: {e}")
        return

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

    # Initialize model based on model_name
    if model_name == 'linear_regression':
        model = LinearRegression(**params)
    elif model_name == 'ridge':
        model = Ridge(**params)
    elif model_name == 'lasso':
        model = Lasso(**params)
    else:
        print(f"Unsupported model type: {model_name}")
        return
    
    # Start MLflow tracking
    with mlflow.start_run(run_name=run_name_param):
        # Log model parameters
        mlflow.log_param("model_name", model_name)
        mlflow.log_params(params)
        
        # Train model
        model.fit(X_train, y_train)
        
        # Predictions and evaluation
        predictions = model.predict(X_test)
        mse = mean_squared_error(y_test, predictions)
        r2 = r2_score(y_test, predictions)
        
        # Log metrics
        mlflow.log_metric("mean_squared_error", mse)
        mlflow.log_metric("r2_score", r2)
        
        # Log the model
        mlflow.sklearn.log_model(model, model_name)
        print(f"Logged {model_name} to MLflow with MSE: {mse} and R2 Score: {r2}")

if __name__ == "__main__":
    experiments = [
        {"model_name": "linear_regression", "params": {}, "run_name_param": "default"},
        {"model_name": "ridge", "params": {"alpha": 1.0}, "run_name_param": "ridge_default"},
        {"model_name": "lasso", "params": {"alpha": 0.1}, "run_name_param": "lasso_default"},
    ]
    for exp in experiments:
        train_and_log_experiment(**exp)


2025/01/22 22:12:46 INFO mlflow.tracking.fluent: Experiment with name 'house_price_prediction_experiment_With_regularization' does not exist. Creating a new experiment.
2025/01/22 22:12:50 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Logged linear_regression to MLflow with MSE: 99442853452.60753 and R2 Score: 0.33522004975064035
🏃 View run default at: http://localhost:5000/#/experiments/578991007214028464/runs/73dc3cd4b9464d1785e79d55000d14e6
🧪 View experiment at: http://localhost:5000/#/experiments/578991007214028464


2025/01/22 22:12:54 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Logged ridge to MLflow with MSE: 99443029522.1547 and R2 Score: 0.3352188727177945
🏃 View run ridge_default at: http://localhost:5000/#/experiments/578991007214028464/runs/70fae0ab0510425a8efc07b85c8e68b9
🧪 View experiment at: http://localhost:5000/#/experiments/578991007214028464


2025/01/22 22:12:58 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Logged lasso to MLflow with MSE: 99442859113.23093 and R2 Score: 0.3352200119091182
🏃 View run lasso_default at: http://localhost:5000/#/experiments/578991007214028464/runs/28fa2f9793394e9480beca8ff5a1ca6c
🧪 View experiment at: http://localhost:5000/#/experiments/578991007214028464


In [16]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import mean_squared_error, r2_score
import mlflow
import mlflow.keras
import pandas as pd
from sklearn.model_selection import train_test_split

# Set the MLflow experiment name
mlflow.set_experiment('Deep_learning_house_price_prediction_experiment_With_regularization')
# Load the dataset
df = pd.read_csv('D:/BITS-wiLP/Semester-3/MLOPs/Assignment/ML Model Deployment/ML Model Deployment/data/house_data.csv')
#df = pd.read_csv('house_data.csv')
X = df[['bedrooms', 'bathrooms', 'floors', 'yr_built']]
y = df['price']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Function to build and train the model based on experiment parameters
def train_and_log_experiment(experiment_config):
    model_name = experiment_config["model_name"]
    epochs = experiment_config.get("epochs", 10)
    batch_size = experiment_config.get("batch_size", 32)
    activation_function = experiment_config.get("activation", 'relu')
    units = experiment_config.get("units", [64, 32])
    
    # Build a neural network model with variable configuration
    model = Sequential()
    model.add(Dense(units[0], input_dim=X_train.shape[1], activation=activation_function))
    model.add(Dense(units[1], activation=activation_function))
    model.add(Dense(1))  # Output layer

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Start MLflow run
    with mlflow.start_run(run_name=model_name):
        # Train the model
        model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)

        # Predictions
        predictions = model.predict(X_test)

        # Evaluate the model
        mse = mean_squared_error(y_test, predictions)
        r2 = r2_score(y_test, predictions)

        # Log metrics
        mlflow.log_metric("mean_squared_error", mse)
        mlflow.log_metric("r2_score", r2)
        
        # Log model and parameters
        mlflow.log_param("epochs", epochs)
        mlflow.log_param("batch_size", batch_size)
        mlflow.log_param("activation_function", activation_function)
        mlflow.log_param("units", units)
        
        # Log the model
        mlflow.keras.log_model(model, "neural_network_model")

        print(f"Logged model '{model_name}' with MSE: {mse} and R2 score: {r2}")


if __name__ == "__main__":
    # List of experiments with different configurations
    experiments = [
        {"model_name": "nn_experiment_1", "epochs": 10, "batch_size": 32, "activation": 'relu', "units": [64, 32]},
        {"model_name": "nn_experiment_2", "epochs": 20, "batch_size": 64, "activation": 'relu', "units": [128, 64]},
        {"model_name": "nn_experiment_3", "epochs": 10, "batch_size": 32, "activation": 'tanh', "units": [64, 32]},
        {"model_name": "nn_experiment_4", "epochs": 50, "batch_size": 16, "activation": 'relu', "units": [256, 128]},
    ]
    
    for exp in experiments:
        train_and_log_experiment(exp)


2025/01/22 22:18:48 INFO mlflow.tracking.fluent: Experiment with name 'Deep_learning_house_price_prediction_experiment_With_regularization' does not exist. Creating a new experiment.
C:\Users\prash\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
507/507 ━━━━━━━━━━━━━━━━━━━━ 1s 734us/step - loss: 386450751488.0000
Epoch 2/10
507/507 ━━━━━━━━━━━━━━━━━━━━ 0s 607us/step - loss: 132852514816.0000
Epoch 3/10
507/507 ━━━━━━━━━━━━━━━━━━━━ 0s 642us/step - loss: 119636148224.0000
Epoch 4/10
507/507 ━━━━━━━━━━━━━━━━━━━━ 0s 655us/step - loss: 128226762752.0000
Epoch 5/10
507/507 ━━━━━━━━━━━━━━━━━━━━ 0s 657us/step - loss: 128538329088.0000
Epoch 6/10
507/507 ━━━━━━━━━━━━━━━━━━━━ 0s 707us/step - loss: 137015238656.0000
Epoch 7/10
507/507 ━━━━━━━━━━━━━━━━━━━━ 0s 681us/step - loss: 139407081472.0000
Epoch 8/10
507/507 ━━━━━━━━━━━━━━━━━━━━ 0s 682us/step - loss: 122273546240.0000
Epoch 9/10
507/507 ━━━━━━━━━━━━━━━━━━━━ 0s 711us/step - loss: 130202025984.0000
Epoch 10/10
507/507 ━━━━━━━━━━━━━━━━━━━━ 0s 710us/step - loss: 129250689024.0000
169/169 ━━━━━━━━━━━━━━━━━━━━ 0s 856us/step


2025/01/22 22:18:53 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/01/22 22:18:59 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
C:\Users\prash\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Logged model 'nn_experiment_1' with MSE: 148888409675.7795 and R2 score: 0.004674281353542198
🏃 View run nn_experiment_1 at: http://localhost:5000/#/experiments/210510871788310002/runs/eaf2f3f9cec64f6a887f4a6fcf6c2884
🧪 View experiment at: http://localhost:5000/#/experiments/210510871788310002
Epoch 1/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 1s 790us/step - loss: 385505787904.0000
Epoch 2/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 0s 783us/step - loss: 155707621376.0000
Epoch 3/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 0s 735us/step - loss: 135657684992.0000
Epoch 4/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 0s 768us/step - loss: 122345529344.0000
Epoch 5/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 0s 777us/step - loss: 129965670400.0000
Epoch 6/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 0s 680us/step - loss: 127777415168.0000
Epoch 7/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 0s 697us/step - loss: 131034152960.0000
Epoch 8/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 0s 774us/step - loss: 123771158528.0000
Epoch 9/20
254/254 ━━━━━━━━━━━━━━━━━━━━ 0s 679us/step - loss: 121

2025/01/22 22:19:04 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/01/22 22:19:10 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
C:\Users\prash\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Logged model 'nn_experiment_2' with MSE: 148294338773.77353 and R2 score: 0.00864567206658473
🏃 View run nn_experiment_2 at: http://localhost:5000/#/experiments/210510871788310002/runs/b20f7690568a4e05865a1b5a8781cc10
🧪 View experiment at: http://localhost:5000/#/experiments/210510871788310002
Epoch 1/10
507/507 ━━━━━━━━━━━━━━━━━━━━ 1s 780us/step - loss: 408572133376.0000
Epoch 2/10
507/507 ━━━━━━━━━━━━━━━━━━━━ 0s 740us/step - loss: 415017795584.0000
Epoch 3/10
507/507 ━━━━━━━━━━━━━━━━━━━━ 0s 710us/step - loss: 407052091392.0000
Epoch 4/10
507/507 ━━━━━━━━━━━━━━━━━━━━ 0s 853us/step - loss: 410328760320.0000
Epoch 5/10
507/507 ━━━━━━━━━━━━━━━━━━━━ 0s 742us/step - loss: 409691062272.0000
Epoch 6/10
507/507 ━━━━━━━━━━━━━━━━━━━━ 0s 740us/step - loss: 425045032960.0000
Epoch 7/10
507/507 ━━━━━━━━━━━━━━━━━━━━ 0s 684us/step - loss: 416148815872.0000
Epoch 8/10
507/507 ━━━━━━━━━━━━━━━━━━━━ 0s 702us/step - loss: 419081486336.0000
Epoch 9/10
507/507 ━━━━━━━━━━━━━━━━━━━━ 0s 624us/step - loss: 422

2025/01/22 22:19:15 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/01/22 22:19:21 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
C:\Users\prash\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Logged model 'nn_experiment_3' with MSE: 449636211908.40454 and R2 score: -2.0058383101932384
🏃 View run nn_experiment_3 at: http://localhost:5000/#/experiments/210510871788310002/runs/bac77e58562e4140bab1b7de75c2a029
🧪 View experiment at: http://localhost:5000/#/experiments/210510871788310002
Epoch 1/50
1014/1014 ━━━━━━━━━━━━━━━━━━━━ 1s 746us/step - loss: 241675452416.0000
Epoch 2/50
1014/1014 ━━━━━━━━━━━━━━━━━━━━ 1s 757us/step - loss: 128112476160.0000
Epoch 3/50
1014/1014 ━━━━━━━━━━━━━━━━━━━━ 1s 786us/step - loss: 136451801088.0000
Epoch 4/50
1014/1014 ━━━━━━━━━━━━━━━━━━━━ 1s 746us/step - loss: 124564529152.0000
Epoch 5/50
1014/1014 ━━━━━━━━━━━━━━━━━━━━ 1s 736us/step - loss: 134424780800.0000
Epoch 6/50
1014/1014 ━━━━━━━━━━━━━━━━━━━━ 1s 750us/step - loss: 125189029888.0000
Epoch 7/50
1014/1014 ━━━━━━━━━━━━━━━━━━━━ 1s 754us/step - loss: 122185957376.0000
Epoch 8/50
1014/1014 ━━━━━━━━━━━━━━━━━━━━ 1s 786us/step - loss: 120873615360.0000
Epoch 9/50
1014/1014 ━━━━━━━━━━━━━━━━━━━━ 1s 806u

2025/01/22 22:20:00 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/01/22 22:20:06 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Logged model 'nn_experiment_4' with MSE: 102922635569.57983 and R2 score: 0.3119575497088256
🏃 View run nn_experiment_4 at: http://localhost:5000/#/experiments/210510871788310002/runs/626d6dc6888b4cc799f85dbc1e0a7b96
🧪 View experiment at: http://localhost:5000/#/experiments/210510871788310002
